In [2]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [3]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [4]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

USD


<Prices: Long/short is at 24.000000000000 on 2020-12-01>

In [5]:
from django.db.models import Count

duplicates = (
            Prices.objects.values('date', 'security', 'price')
            .annotate(count_id=Count('id'))
            .filter(count_id__gt=1)
        )

total_duplicates = len(duplicates)
print((f"Found {total_duplicates} duplicate entries."))

for i, duplicate in enumerate(duplicates, start=1):
            # Keep one entry and delete the rest
            entries = Prices.objects.filter(
                date=duplicate['date'],
                security=duplicate['security'],
                price=duplicate['price']
            )
            entries_to_keep = entries.first()
            entries.exclude(id=entries_to_keep.id).delete()

            # Print status update
            print(f"Duplicates are being removed: {i} out of {total_duplicates}\r", end='', flush=True)
            
print("\nDuplicates removed.")


Found 0 duplicate entries.

Duplicates removed.


In [30]:
import datetime
from decimal import Decimal


def end_of_year_price_correction(user, year, broker_name, target_nav, asset_name):
    
    target_nav = round(Decimal(target_nav), 2)
    
    # Get the broker
    try:
        broker = Brokers.objects.get(name=broker_name)
    except Brokers.DoesNotExist:
        return {"error": f"Broker {broker_name} does not exist."}

    # Get the asset
    try:
        asset = Assets.objects.get(name=asset_name)
    except Assets.DoesNotExist:
        return {"error": f"Asset {asset_name} does not exist."}

    # Calculate end of year date
    end_of_year_date = datetime.date(year, 12, 31)

    # Fetch NAV at the end of the year
    # nav_at_end_of_year = get_nav_at_date(broker, end_of_year_date)
    nav_at_end_of_year = NAV_at_date(user.id, [broker.id], end_of_year_date, asset.currency, [])['Total NAV']
    if nav_at_end_of_year is None:
        return {"error": f"No NAV found for broker {broker_name} at the end of {year}."}

    # Fetch asset price and position at the end of the year
    price_at_end_of_year = asset.price_at_date(end_of_year_date)
    if not price_at_end_of_year:
        return {"error": f"No price found for asset {asset_name} at the end of {year}."}

    position_at_end_of_year = asset.position(end_of_year_date, [broker.id])

    # Calculate new price
    old_price = price_at_end_of_year.price
    new_price = old_price + ((target_nav - nav_at_end_of_year) / position_at_end_of_year)

    # Display information
    old_asset_value = round(Decimal(old_price * position_at_end_of_year), 2)
    new_asset_value = round(Decimal(new_price * position_at_end_of_year), 2)

    result = {
        "old_price": old_price,
        "new_price": new_price,
        "old_asset_value": old_asset_value,
        "new_asset_value": new_asset_value,
        "old_nav": nav_at_end_of_year,
        "target_nav": target_nav
    }

    print(f"Old Price: {result['old_price']}, New Price: {result['new_price']}")
    print(f"Old Asset Value: {result['old_asset_value']}, New Asset Value: {result['new_asset_value']}")
    print(f"Old NAV: {result['old_nav']}, Target NAV: {result['target_nav']}")

    # Ask for confirmation
    confirm = input("Do you want to update the price? (yes/no): ")

    if confirm.lower() == 'yes':
        # Update the price
        price_instance, created = Prices.objects.get_or_create(
            security=asset, date=end_of_year_date,
            defaults={'price': new_price}
        )
        if not created:
            price_instance.price = new_price
            price_instance.save()

        result["status"] = "Price updated successfully."
        print("New price is:", price_instance.price)
    else:
        result["status"] = "Price update canceled."

    return result

user = CustomUser.objects.filter(username='Yaroslav').first()
year = 2013
broker_name = 'UBS Pension'
target_nav = 29444.38
asset_name = 'Emerging Markets Equity Fund'

end_of_year_price_correction(user, year, broker_name, target_nav, asset_name)

Old Price: 0.936630, New Price: 0.9366299527568694238659180266
Old Asset Value: 7983.27, New Asset Value: 7983.27
Old NAV: 29444.380402671861, Target NAV: 29444.38


{'old_price': Decimal('0.936630'),
 'new_price': Decimal('0.9366299527568694238659180266'),
 'old_asset_value': Decimal('7983.27'),
 'new_asset_value': Decimal('7983.27'),
 'old_nav': Decimal('29444.380402671861'),
 'target_nav': Decimal('29444.38'),
 'status': 'Price update canceled.'}

In [35]:
from django.db import IntegrityError
import pandas as pd

def import_asset_prices_from_csv(file_path, investor_id):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Convert the 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

    # Get column names (asset names) excluding 'date'
    asset_names = [col for col in df.columns if col != 'date']
    print(asset_names)

    # Create a dictionary to store asset_name to asset_id mapping
    asset_name_to_id = {}

    # Check if assets exist for the investor and create the mapping
    for asset_name in asset_names:
        try:
            asset = Assets.objects.get(investor_id=investor_id, name=asset_name)
            asset_name_to_id[asset_name] = asset.id
        except Assets.DoesNotExist:
            print(f"Warning: Asset '{asset_name}' not found for investor {investor_id}")

    # Iterate through the DataFrame
    for index, row in df.iterrows():
        date = row['date'].date()
        
        # Iterate through each asset column
        for asset_name in asset_names:
            price = row[asset_name]
            
            if pd.isna(price):
                continue
            
            asset_id = asset_name_to_id.get(asset_name)
            if asset_id is None:
                continue  # We've already printed a warning, so just skip this asset

            price_data = {
                'date': date,
                'security_id': asset_id,
                'price': Decimal(str(price)),
            }

            # print(price_data)

            # Update or create the Price entry
            try:
                price_instance, created = Prices.objects.update_or_create(
                    date=price_data['date'], 
                    security_id=price_data['security_id'],
                    defaults=price_data
                )
                print(f"{'Created' if created else 'Updated'} price for asset {asset_name} on date {date}")
            except IntegrityError as e:
                print(f"Error updating price for asset {asset_name} on date {date}: {e}")


import_asset_prices_from_csv('../Data/Pricing data from Galaxy.csv', 2)

['UBS Europe (ex-UK) Equity Tracker', 'UBS Pacific (exJapan) Equity Tracker', 'UBS UK Equity Tracker', 'Global Listed Property Tracker Fund', 'North America Equity Tracker Fund', 'World (ex-UK) Equity Tracker Fund', 'Growth Fund']
Updated price for asset UBS Europe (ex-UK) Equity Tracker on date 2021-12-31
Updated price for asset UBS Pacific (exJapan) Equity Tracker on date 2021-12-31
Updated price for asset UBS UK Equity Tracker on date 2021-12-31
Updated price for asset Global Listed Property Tracker Fund on date 2021-12-31
Updated price for asset North America Equity Tracker Fund on date 2021-12-31
Updated price for asset World (ex-UK) Equity Tracker Fund on date 2021-12-31
Updated price for asset Growth Fund on date 2021-12-31
Created price for asset UBS Europe (ex-UK) Equity Tracker on date 2022-12-31
Created price for asset UBS Pacific (exJapan) Equity Tracker on date 2022-12-31
Created price for asset UBS UK Equity Tracker on date 2022-12-31
Created price for asset Global Listed